In [1]:
# !pip install timm einops mmcv &> /dev/null

In [2]:
# import sys

# ! git clone https://github.com/shoaib6174/GSOC-22-Video-Swin-Transformers
# sys.path.append('/content/GSOC-22-Video-Swin-Transformers')

In [3]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict

torch.set_grad_enabled(False)

# Helper function


In [4]:
def get_x(shape=(2,3,8,224,224)):
    x_pt = torch.rand(shape)
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt
    

In [5]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
del cfg["drop_path_rate"]
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

In [6]:
# load pt model with weights
pt_model = SwinTransformer3D_pt(**cfg)
x_tf, x_pt = get_x()

_= pt_model(x_pt)

# checkpoint = torch.load(f'swin_tiny_patch244_window877_kinetics400_1k.pth')
# new_state_dict = OrderedDict()
# for k, v in checkpoint['state_dict'].items():
#     if 'backbone' in k:
#         name = k[9:]
#         new_state_dict[name] = v 

# pt_model.load_state_dict(new_state_dict) 


/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2022-09-03 21:49:17.369407: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-03 21:49:17.369518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB


 
basic layer call torch.Size([2, 96, 4, 56, 56])
forward path (4, 56, 56) (8, 7, 7) (0, 0, 0)
forward path (4, 56, 56) (8, 7, 7) (4, 3, 3)

 
basic layer call torch.Size([2, 192, 4, 28, 28])
forward path (4, 28, 28) (8, 7, 7) (0, 0, 0)
forward path (4, 28, 28) (8, 7, 7) (4, 3, 3)

 
basic layer call torch.Size([2, 384, 4, 14, 14])
forward path (4, 14, 14) (8, 7, 7) (0, 0, 0)
forward path (4, 14, 14) (8, 7, 7) (4, 3, 3)
forward path (4, 14, 14) (8, 7, 7) (0, 0, 0)
forward path (4, 14, 14) (8, 7, 7) (4, 3, 3)
forward path (4, 14, 14) (8, 7, 7) (0, 0, 0)
forward path (4, 14, 14) (8, 7, 7) (4, 3, 3)

 
basic layer call torch.Size([2, 768, 4, 7, 7])
forward path (4, 7, 7) (8, 7, 7) (0, 0, 0)
forward path (4, 7, 7) (8, 7, 7) (4, 3, 3)


In [7]:
#@title Convert functions
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    
def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False, **kwargs):
    in_shape = pt_weight.shape
    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)
    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()
    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))
        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))
    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
    return tf_component

def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


In [8]:
#@title modify_swin_block3D

def modify_swin_block3D(np_state_dict, pt_weights_prefix, tf_block):
    layernorm_idx = 1
    mlp_layer_idx = 1


    if isinstance(tf_block, SwinTransformerBlock3D):
        print(tf_block)
        for inner_layer in tf_block.layers:
            
            # Layer norm.
            if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                layer_norm_prefix = (
                    f"norm{layernorm_idx}"
                )
                inner_layer.gamma.assign(
                    tf.Variable(
                        np_state_dict[f"{layer_norm_prefix}.weight"]
                    )
                )



                inner_layer.beta.assign(
                    tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                )

                layernorm_idx += 1

            # Window attention.
            elif isinstance(inner_layer, WindowAttention3D):
                attn_prefix = f"attn"

                # Relative position.
                inner_layer.relative_position_bias_table = (
                    modify_tf_block(
                        inner_layer.relative_position_bias_table,
                        np_state_dict[
                            f"{attn_prefix}.relative_position_bias_table"
                        ] 
                    )
                )
                inner_layer.relative_position_index = (
                    modify_tf_block(
                        inner_layer.relative_position_index,
                        np_state_dict[
                            f"{attn_prefix}.relative_position_index"
                        ]
                    )
                )

                # QKV.
                inner_layer.qkv = modify_tf_block(
                    inner_layer.qkv,
                    np_state_dict[f"{attn_prefix}.qkv.weight"],
                    np_state_dict[f"{attn_prefix}.qkv.bias"]
                )

                # Projection.
                inner_layer.proj = modify_tf_block(
                    inner_layer.proj,
                    np_state_dict[f"{attn_prefix}.proj.weight"],
                    np_state_dict[f"{attn_prefix}.proj.bias"]
                )

            # MLP.
            elif isinstance(inner_layer, tf.keras.Model):
                mlp_prefix = f"mlp"
                for mlp_layer in inner_layer.layers:
                    if isinstance(mlp_layer, tf.keras.layers.Dense):
                        mlp_layer = modify_tf_block(
                            mlp_layer,
                            np_state_dict[
                                f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                            ],
                            np_state_dict[
                                f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                            ]
                        )
                        mlp_layer_idx += 1

        
    return tf_block

In [9]:
def compute_mask(D, H, W, window_size, shift_size):

    img_mask = np.zeros((1, D, H, W, 1)) 

 
    cnt = 0

    for d in slice(-window_size[0]), slice(-window_size[0], -shift_size[0]), slice(-shift_size[0],None):
        for h in slice(-window_size[1]), slice(-window_size[1], -shift_size[1]), slice(-shift_size[1],None):
            for w in slice(-window_size[2]), slice(-window_size[2], -shift_size[2]), slice(-shift_size[2],None):
                img_mask[:, d, h, w, :] = cnt
                cnt = cnt + 1
    img_mask = tf.convert_to_tensor(img_mask, dtype="float32")

    # print("basic compute", img_mask.shape, window_size, shift_size, D, H, W)

    mask_windows = window_partition(img_mask, window_size)  # nW, ws[0]*ws[1]*ws[2], 1

    mask_windows = tf.squeeze(mask_windows, axis = -1)  # nW, ws[0]*ws[1]*ws[2] ??
    attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(mask_windows, axis=2)
    

    attn_mask = tf.cast(attn_mask, dtype="float64")

    attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
    attn_mask = tf.where(attn_mask == 0, 0.0 , attn_mask)

    return attn_mask


In [10]:
#@title pt model


'''
Credit to the official implementation: https://github.com/SwinTransformer/Video-Swin-Transformer
'''


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
import numpy as np
from timm.models.layers import  trunc_normal_
from timm.models.layers import DropPath as DropPath_pt
from functools import reduce, lru_cache
from operator import mul
from einops import rearrange

import logging
from mmcv.utils import get_logger
from mmcv.runner import load_checkpoint


def get_root_logger_pt(log_file=None, log_level=logging.INFO):
    """Use ``get_logger`` method in mmcv to get the root logger.
    The logger will be initialized if it has not been initialized. By default a
    StreamHandler will be added. If ``log_file`` is specified, a FileHandler
    will also be added. The name of the root logger is the top-level package
    name, e.g., "mmaction".
    Args:
        log_file (str | None): The log filename. If specified, a FileHandler
            will be added to the root logger.
        log_level (int): The root logger level. Note that only the process of
            rank 0 is affected, while other processes will set the level to
            "Error" and be silent most of the time.
    Returns:
        :obj:`logging.Logger`: The root logger.
    """
    return get_logger(__name__.split('.')[0], log_file, log_level)


class Mlp_pt(nn.Module):
    """ Multilayer perceptron."""

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        # print('dense',hidden_features, in_features)

        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        # print("x", x.shape)

        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


def window_partition_pt(x, window_size):
    """
    Args:
        x: (B, D, H, W, C)
        window_size (tuple[int]): window size
    Returns:
        windows: (B*num_windows, window_size*window_size, C)
    """
    print("window_partition", x.shape, window_size)
    B, D, H, W, C = x.shape
    x = x.view(B, D // window_size[0], window_size[0], H // window_size[1], window_size[1], W // window_size[2], window_size[2], C)
    windows = x.permute(0, 1, 3, 5, 2, 4, 6, 7).contiguous().view(-1, reduce(mul, window_size), C)
    return windows


def window_reverse_pt(windows, window_size, B, D, H, W):
    """
    Args:
        windows: (B*num_windows, window_size, window_size, C)
        window_size (tuple[int]): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, D, H, W, C)
    """
    x = windows.view(B, D // window_size[0], H // window_size[1], W // window_size[2], window_size[0], window_size[1], window_size[2], -1)
    x = x.permute(0, 1, 4, 2, 5, 3, 6, 7).contiguous().view(B, D, H, W, -1)
    return x




def get_window_size_pt(x_size, window_size, shift_size=None):
    use_window_size = list(window_size)
    if shift_size is not None:
        use_shift_size = list(shift_size)
    for i in range(len(x_size)):
        if x_size[i] <= window_size[i]:
            use_window_size[i] = x_size[i]
            if shift_size is not None:
                use_shift_size[i] = 0

    if shift_size is None:
        return tuple(use_window_size)
    else:
        return tuple(use_window_size), tuple(use_shift_size)


class WindowAttention3D_pt(nn.Module):
    """ Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The temporal length, height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wd, Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1) * (2 * window_size[2] - 1), num_heads))  # 2*Wd-1 * 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_d = torch.arange(self.window_size[0])
        coords_h = torch.arange(self.window_size[1])
        coords_w = torch.arange(self.window_size[2])
        coords = torch.stack(torch.meshgrid(coords_d, coords_h, coords_w))  # 3, Wd, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 3, Wd*Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 3, Wd*Wh*Ww, Wd*Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wd*Wh*Ww, Wd*Wh*Ww, 3
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 2] += self.window_size[2] - 1

        relative_coords[:, :, 0] *= (2 * self.window_size[1] - 1) * (2 * self.window_size[2] - 1)
        relative_coords[:, :, 1] *= (2 * self.window_size[2] - 1)
        relative_position_index = relative_coords.sum(-1)  # Wd*Wh*Ww, Wd*Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """ Forward function.
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, N, N) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # B_, nH, N, C

        q = q * self.scale
        attn = q @ k.transpose(-2, -1)

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index[:N, :N].reshape(-1)].reshape(
            N, N, -1)  # Wd*Wh*Ww,Wd*Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wd*Wh*Ww, Wd*Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0) # B_, nH, N, N

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class SwinTransformerBlock3D_pt(nn.Module):

    def __init__(self, dim, num_heads, window_size=(2,7,7), shift_size=(0,0,0),
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm, use_checkpoint=False):
        super().__init__()

        print("3d block",  dim, num_heads, window_size, shift_size,
                 mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path,
                 act_layer, norm_layer, use_checkpoint)
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        self.use_checkpoint=use_checkpoint

        assert 0 <= self.shift_size[0] < self.window_size[0], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[1] < self.window_size[1], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[2] < self.window_size[2], "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention3D_pt(
            dim, window_size=self.window_size, num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath_pt(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp_pt(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.outputs = {}
    def forward_part1(self, x, mask_matrix):
        B, D, H, W, C = x.shape
        window_size, shift_size = get_window_size_pt((D, H, W), self.window_size, self.shift_size)
        x = self.norm1(x)
        # pad feature maps to multiples of window size
        pad_l = pad_t = pad_d0 = 0
        pad_d1 = (window_size[0] - D % window_size[0]) % window_size[0]
        pad_b = (window_size[1] - H % window_size[1]) % window_size[1]
        pad_r = (window_size[2] - W % window_size[2]) % window_size[2]
        x = F.pad(x, (0, 0, pad_l, pad_r, pad_t, pad_b, pad_d0, pad_d1))

        self.outputs["paddings"] = x

        _, Dp, Hp, Wp, _ = x.shape
        # cyclic shift
        if any(i > 0 for i in shift_size):
            shifted_x = torch.roll(x, shifts=(-shift_size[0], -shift_size[1], -shift_size[2]), dims=(1, 2, 3))
            attn_mask = mask_matrix
        else:
            shifted_x = x
            attn_mask = None
        

        self.outputs["shifted_x"] = shifted_x
        self.outputs["attn_mask"] = attn_mask
        # partition windows
        x_windows = window_partition_pt(shifted_x, window_size)  # B*nW, Wd*Wh*Ww, C
        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=attn_mask)  # B*nW, Wd*Wh*Ww, C
        # merge windows
        attn_windows = attn_windows.view(-1, *(window_size+(C,)))
        shifted_x = window_reverse_pt(attn_windows, window_size, B, Dp, Hp, Wp)  # B D' H' W' C

        self.outputs["shifted_x final"] = shifted_x
        self.outputs["attn_windows"] = attn_windows
        self.outputs["x_windowns"] = x_windows
        # reverse cyclic shift
        if any(i > 0 for i in shift_size):
            x = torch.roll(shifted_x, shifts=(shift_size[0], shift_size[1], shift_size[2]), dims=(1, 2, 3))
        else:
            x = shifted_x

        self.outputs["after roll"] = x
        

        if pad_d1 >0 or pad_r > 0 or pad_b > 0:
            x = x[:, :D, :H, :W, :].contiguous()

        self.outputs["forward1 output"] = x
        
        return x

    def forward_part2(self, x):
        x = self.norm2(x)
        self.outputs["norm2"] =x
        x = self.mlp(x)
        self.outputs["mlp"] = x
        x= self.drop_path(x)
        self.outputs['drop_path'] = x
        return x
        # return self.drop_path(self.mlp(self.norm2(x)))

    def forward(self, x, mask_matrix):
        shortcut = x
        if self.use_checkpoint:
            x = checkpoint.checkpoint(self.forward_part1, x, mask_matrix)
        else:
            x = self.forward_part1(x, mask_matrix)
        # self.outputs['forward_part1'] = x

        x = shortcut + self.drop_path(x)
        # self.outputs['after shortcut'] = x

        if self.use_checkpoint:
            x = x + checkpoint.checkpoint(self.forward_part2, x)
        else:
            x = x + self.forward_part2(x)
        # self.outputs['forward_part2'] = x
        

        return x, self.outputs


class PatchMerging_pt(nn.Module):
    """ Patch Merging Layer
    Args:
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """
    def __init__(self, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """ Forward function.
        Args:
            x: Input feature, tensor size (B, D, H, W, C).
        """
        B, D, H, W, C = x.shape

        # padding
        pad_input = (H % 2 == 1) or (W % 2 == 1)
        if pad_input:
            x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))

        x0 = x[:, :, 0::2, 0::2, :]  # B D H/2 W/2 C
        x1 = x[:, :, 1::2, 0::2, :]  # B D H/2 W/2 C
        x2 = x[:, :, 0::2, 1::2, :]  # B D H/2 W/2 C
        x3 = x[:, :, 1::2, 1::2, :]  # B D H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B D H/2 W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x


# cache each stage results
def compute_mask_pt(D, H, W, window_size, shift_size, device):
    img_mask = torch.zeros((1, D, H, W, 1), device=device)  # 1 Dp Hp Wp 1
    cnt = 0
    for d in slice(-window_size[0]), slice(-window_size[0], -shift_size[0]), slice(-shift_size[0],None):
        for h in slice(-window_size[1]), slice(-window_size[1], -shift_size[1]), slice(-shift_size[1],None):
            for w in slice(-window_size[2]), slice(-window_size[2], -shift_size[2]), slice(-shift_size[2],None):
                img_mask[:, d, h, w, :] = cnt
                cnt += 1
    mask_windows = window_partition_pt(img_mask, window_size)  # nW, ws[0]*ws[1]*ws[2], 1
    mask_windows = mask_windows.squeeze(-1)  # nW, ws[0]*ws[1]*ws[2]
    attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
    attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
    return attn_mask


class BasicLayer_pt(nn.Module):

    def __init__(self,dim,depth,num_heads,window_size=(1,7,7), mlp_ratio=4.,qkv_bias=False,
                 qk_scale=None,drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 norm_layer=nn.LayerNorm,
                 downsample=None,
                 use_checkpoint=False):
        super().__init__()
        # print(self, dim,depth,num_heads,window_size, mlp_ratio,qkv_bias,qk_scale,drop,attn_drop,drop_path,norm_layer,downsample,use_checkpoint )

        self.window_size = window_size
        self.shift_size = tuple(i // 2 for i in window_size)
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock3D_pt(
                dim=dim,
                num_heads=num_heads,
                window_size=window_size,
                shift_size=(0,0,0) if (i % 2 == 0) else self.shift_size,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop,
                attn_drop=attn_drop,
                drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                norm_layer=norm_layer,
                use_checkpoint=use_checkpoint,
            )
            for i in range(depth)])
        
        self.downsample = downsample
        if self.downsample is not None:
            self.downsample = downsample(dim=dim, norm_layer=norm_layer)

        self.outputs= {}
    def forward(self, x):
        """ Forward function.
        Args:
            x: Input feature, tensor size (B, C, D, H, W).
        """
        # calculate attention mask for SW-MSA
        B, C, D, H, W = x.shape
        window_size, shift_size = get_window_size_pt((D,H,W), self.window_size, self.shift_size)
        x = rearrange(x, 'b c d h w -> b d h w c')
        Dp = int(np.ceil(D / window_size[0])) * window_size[0]
        Hp = int(np.ceil(H / window_size[1])) * window_size[1]
        Wp = int(np.ceil(W / window_size[2])) * window_size[2]
        attn_mask = compute_mask_pt(Dp, Hp, Wp, window_size, shift_size, x.device)
        i = 1
        self.outputs["attn_mask"] = attn_mask

        for blk in self.blocks:
            x , _= blk(x, attn_mask)
            self.outputs[f"block_{i}"] = x
            i += 1


        x = x.view(B, D, H, W, -1)

        if self.downsample is not None:
            x = self.downsample(x)
        x = rearrange(x, 'b d h w c -> b c d h w')

        
        return x ,self.outputs


class PatchEmbed3D_pt(nn.Module):

    def __init__(self, patch_size=(2,4,4), in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        self.patch_size = patch_size

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv3d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        """Forward function."""
        # padding
        _, _, D, H, W = x.size()
        if W % self.patch_size[2] != 0:
            x = F.pad(x, (0, self.patch_size[2] - W % self.patch_size[2]))
        if H % self.patch_size[1] != 0:
            x = F.pad(x, (0, 0, 0, self.patch_size[1] - H % self.patch_size[1]))
        if D % self.patch_size[0] != 0:
            x = F.pad(x, (0, 0, 0, 0, 0, self.patch_size[0] - D % self.patch_size[0]))

        x = self.proj(x)  # B C D Wh Ww
        if self.norm is not None:
            D, Wh, Ww = x.size(2), x.size(3), x.size(4)
            x = x.flatten(2).transpose(1, 2)
            x = self.norm(x)
            x = x.transpose(1, 2).view(-1, self.embed_dim, D, Wh, Ww)

        return x


class SwinTransformer3D_pt(nn.Module):


    def __init__(self,
                 pretrained=None,
                 pretrained2d=True,
                 patch_size=(4,4,4),
                 in_chans=3,
                 embed_dim=96,
                 depths=[2, 2, 6, 2],
                 num_heads=[3, 6, 12, 24],
                 window_size=(2,7,7),
                 mlp_ratio=4.,
                 qkv_bias=True,
                 qk_scale=None,
                 drop_rate=0.,
                 attn_drop_rate=0.,
                 drop_path_rate=0.2,
                 norm_layer=nn.LayerNorm,
                 patch_norm=False,
                 frozen_stages=-1,
                 use_checkpoint=False, 
                 isTest =False):    ## ****** remove it later
        super().__init__()

        self.pretrained = pretrained
        self.pretrained2d = pretrained2d
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.patch_norm = patch_norm
        self.frozen_stages = frozen_stages
        self.window_size = window_size
        self.patch_size = patch_size

        self.isTest = isTest

        # split image into non-overlapping patches
        self.patch_embed = PatchEmbed3D_pt(
            patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None)

        self.pos_drop = nn.Dropout(p=drop_rate)

        # stochastic depth
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]  # stochastic depth decay rule

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer_pt(
                dim=int(embed_dim * 2**i_layer),
                depth=depths[i_layer],
                num_heads=num_heads[i_layer],
                window_size=window_size,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[sum(depths[:i_layer]):sum(depths[:i_layer + 1])],
                norm_layer=norm_layer,
                downsample=PatchMerging_pt if i_layer<self.num_layers-1 else None,
                use_checkpoint=use_checkpoint)
            self.layers.append(layer)

        self.num_features = int(embed_dim * 2**(self.num_layers-1))

        # add a norm layer for each output
        self.norm = norm_layer(self.num_features)

        self._freeze_stages()

    def _freeze_stages(self):
        if self.frozen_stages >= 0:
            self.patch_embed.eval()
            for param in self.patch_embed.parameters():
                param.requires_grad = False

        if self.frozen_stages >= 1:
            self.pos_drop.eval()
            for i in range(0, self.frozen_stages):
                m = self.layers[i]
                m.eval()
                for param in m.parameters():
                    param.requires_grad = False


    def forward(self, x):
        """Forward function."""
        x = self.patch_embed(x)
        layer_out = {}

        layer_out["PatchEmbed"] = x
        x = self.pos_drop(x)

        i = 1

        for layer in self.layers:
            x = layer(x.contiguous())
            layer_out[f"basic layer{i}"] = x
            i+=1

        x = rearrange(x, 'n c d h w -> n d h w c')
        x = self.norm(x)
        x = rearrange(x, 'n d h w c -> n c d h w')
        
        layer_out["Final Output"] = x

        if self.isTest:                 # remove later
            return layer_out, x
        else:
            return x

    def train(self, mode=True):
        """Convert the model into training mode while keep layers freezed."""
        super(SwinTransformer3D_pt, self).train(mode)
        self._freeze_stages()

# Swin Block3d (b4,3d_2)

In [21]:

import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization
import numpy as np



from functools import  lru_cache
def compute_mask(D, H, W, window_size, shift_size):

    img_mask = np.zeros((1, D, H, W, 1)) 

 
    cnt = 0

    for d in slice(-window_size[0]), slice(-window_size[0], -shift_size[0]), slice(-shift_size[0],None):
        for h in slice(-window_size[1]), slice(-window_size[1], -shift_size[1]), slice(-shift_size[1],None):
            for w in slice(-window_size[2]), slice(-window_size[2], -shift_size[2]), slice(-shift_size[2],None):
                img_mask[:, d, h, w, :] = cnt
                cnt = cnt + 1
    img_mask = tf.convert_to_tensor(img_mask, dtype="float32")

    # print("basic compute", img_mask.shape, window_size, shift_size, D, H, W)
    # print("compute mask cm tf")
    mask_windows = window_partition(img_mask, window_size)  # nW, ws[0]*ws[1]*ws[2], 1

    mask_windows = tf.squeeze(mask_windows, axis = -1)  # nW, ws[0]*ws[1]*ws[2] ??
    attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(mask_windows, axis=2)
    

    attn_mask = tf.cast(attn_mask, dtype="float64")

    attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
    attn_mask = tf.where(attn_mask == 0, 0.0 , attn_mask)

    return attn_mask


class SwinTransformerBlock3D(tf.keras.Model):


    def __init__(self, dim, compute_mask_info, num_heads, window_size=(2,7,7), shift_size=(0,0,0),
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=tf.keras.activations.gelu, norm_layer=LayerNormalization, use_checkpoint=False):
        super().__init__()

        print("3d block",  dim, compute_mask_info,  num_heads, window_size, shift_size,
                 mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path,
                 act_layer, norm_layer, use_checkpoint)

        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        self.use_checkpoint=use_checkpoint
        self.compute_mask_info = compute_mask_info
        

        # delete this
        self.drop_path_val = drop_path
 
        assert 0 <= self.shift_size[0] < self.window_size[0], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[1] < self.window_size[1], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[2] < self.window_size[2], "shift_size must in 0-window_size"

        self.norm1 = norm_layer( epsilon=1e-5)
        self.attn = WindowAttention3D(
            dim, window_size=self.window_size, num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        # compute mask
        B, C, D, H, W = self.compute_mask_info["shape_of_input"]
        mask_window_size, mask_shift_size = get_window_size((D,H,W), self.compute_mask_info["window_size"], self.compute_mask_info["shift_size"])  #### change
         
        Dp = int(tf.math.ceil(D/ mask_window_size[0])) * mask_window_size[0]
        Hp = int(tf.math.ceil(H / mask_window_size[1])) * mask_window_size[1]
        Wp = int(tf.math.ceil(W / mask_window_size[2])) * mask_window_size[2]
        
        self.attn_mask = compute_mask(Dp, Hp, Wp, mask_window_size, mask_shift_size)
        # print("compute mask parameters", (Dp, Hp, Wp, mask_window_size, mask_shift_size))
        # print("attn_mask", self.attn_mask.shape)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else tf.identity
        self.norm2 = norm_layer(epsilon=1e-5)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = mlp_block(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.outputs = {}

    def forward_part1(self, x):
        
        B, D, H, W, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2] , tf.shape(x)[3] , tf.shape(x)[4] 
        
        b, c, d, h ,w = self.compute_mask_info['shape_of_input']
        

        window_size, shift_size = get_window_size((d, h, w), self.window_size, self.shift_size)
        print("get window size", window_size, shift_size, (d, h, w), self.window_size, self.shift_size, (D, H,W))

        x = self.norm1(x)

        pad_l = pad_t = pad_d0 = 0
 
        pad_d1 = (window_size[0] - d % window_size[0]) % window_size[0]
        pad_b = (window_size[1] - h % window_size[1]) % window_size[1]
        pad_r = (window_size[2] - w % window_size[2]) % window_size[2]

  

        paddings = [[0,0] , [pad_d0, pad_d1] , [pad_t, pad_b] , [pad_l, pad_r], [0, 0] ]


        x = tf.pad(x, paddings)
        self.outputs["paddings"] = x
        
        _, Dp, Hp, Wp, _ = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2] , tf.shape(x)[3] , tf.shape(x)[4] 

        if any(i > 0 for i in shift_size):
            shifted_x = tf.roll(x, shift=[-shift_size[0], -shift_size[1], -shift_size[2]], axis=[1, 2, 3]) #?
            attn_mask = self.attn_mask
        else:
            shifted_x = x
            attn_mask = None

        self.outputs["shifted_x"] = shifted_x
        self.outputs["attn_mask"] = attn_mask

        x_windows = window_partition(shifted_x, window_size)  # B*nW, Wd*Wh*Ww, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=attn_mask)  # B*nW, Wd*Wh*Ww, C
        # merge windows
        attn_windows = tf.reshape( attn_windows ,  [-1, *(window_size+(C,))] )
        shifted_x = window_reverse(attn_windows, window_size, B, Dp, Hp, Wp)  # B D' H' W' C

        self.outputs["shifted_x final"] = shifted_x
        self.outputs["attn_windows"] = attn_windows
        self.outputs["x_windowns"] = x_windows
        print("before roll", shift_size)
        if any(i > 0 for i in shift_size):
            x = tf.roll(shifted_x, shift=[shift_size[0], shift_size[1], shift_size[2]], axis=[1, 2, 3]) #?
        else:
            x = shifted_x

        self.outputs["after roll"] = x

        if pad_d1 >0 or pad_r > 0 or pad_b > 0:
            x = x[:, :D, :H, :W, :]
        self.outputs["forward1 output"] = x
        return x

    def forward_part2(self, x):
        x = self.norm2(x)
        self.outputs["norm2"] =x
        x = self.mlp(x)
        self.outputs["mlp"] = x
        x= self.drop_path(x)
        self.outputs['drop_path'] = x
        return x


    def call(self, x):


        shortcut = x
        x = self.forward_part1(x)

        self.outputs['forward_part1'] = x

        x = shortcut + self.drop_path(x)
        self.outputs['after shortcut'] = x

        x = x + self.forward_part2(x)
        
        self.outputs['forward_part2'] = x

        return x, self.outputs

In [22]:
b4_3d_pt = SwinTransformerBlock3D_pt(  768, 24, (8, 7, 7), (4, 3, 3), 4.0, True, None, 0.4, 0.0, 0.0, nn.GELU, nn.LayerNorm, False)
b4_3d_tf = SwinTransformerBlock3D(  768, {'shape_of_input': (2, 768, 16, 7, 7), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)} , 24, (8, 7, 7), (4, 3, 3), 4.0, True, None, 0.4, 0.0, 0.0, tf.keras.activations.gelu, tf.keras.layers.LayerNormalization, False)


3d block 768 24 (8, 7, 7) (4, 3, 3) 4.0 True None 0.4 0.0 0.0 <class 'torch.nn.modules.activation.GELU'> <class 'torch.nn.modules.normalization.LayerNorm'> False
3d block 768 {'shape_of_input': (2, 768, 16, 7, 7), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)} 24 (8, 7, 7) (4, 3, 3) 4.0 True None 0.4 0.0 0.0 <function gelu at 0x165b2b700> <class 'keras.layers.normalization.layer_normalization.LayerNormalization'> False
window_partition (1, 16, 7, 7, 1) (8, 7, 7)


In [23]:
mask = compute_mask_pt(16, 7, 7, (8, 7, 7), (4, 0, 0), None)
mask.shape
# 16 7 7 (8, 7, 7) (4, 0, 0)

window_partition torch.Size([1, 16, 7, 7, 1]) (8, 7, 7)


torch.Size([2, 392, 392])

In [24]:
x_tf , x_pt = get_x((2, 16, 7, 7, 768))
out_pt, layers_3d_pt = b4_3d_pt(x_pt, mask)
out_tf, layers_3d_pt = b4_3d_tf(x_tf)


out_tf[:1,:1,:1,:1,:10], out_pt[:1,:1,:1,:1,:10]

window_partition torch.Size([2, 16, 7, 7, 768]) (8, 7, 7)
get window size (8, 7, 7) (4, 0, 0) (16, 7, 7) (8, 7, 7) (4, 3, 3) (<tf.Tensor: shape=(), dtype=int32, numpy=16>, <tf.Tensor: shape=(), dtype=int32, numpy=7>, <tf.Tensor: shape=(), dtype=int32, numpy=7>)
window_partition (2, 16, 7, 7, 768) (8, 7, 7)
before roll (4, 0, 0)


(<tf.Tensor: shape=(1, 1, 1, 1, 10), dtype=float32, numpy=
 array([[[[[ 1.0243059 , -0.30789116, -0.6846147 ,  0.35682264,
             0.9671856 , -0.29690248,  1.3214772 ,  0.02924263,
             0.15522276,  0.11837676]]]]], dtype=float32)>,
 tensor([[[[[ 0.6372,  0.2080,  0.6560, -0.1802, -0.2434,  0.1458,  0.6642,
             -0.0605,  0.2814,  0.3322]]]]]))

In [25]:
b4_3d_pt.eval()

np_state_dict = b4_3d_pt.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}
np_state_dict.keys()

dict_keys(['norm1.weight', 'norm1.bias', 'attn.relative_position_bias_table', 'attn.relative_position_index', 'attn.qkv.weight', 'attn.qkv.bias', 'attn.proj.weight', 'attn.proj.bias', 'norm2.weight', 'norm2.bias', 'mlp.fc1.weight', 'mlp.fc1.bias', 'mlp.fc2.weight', 'mlp.fc2.bias'])

In [26]:
model_layers = {}
def modify_block3D(np_state_dict, pt_weights_prefix, outer_layer):
  # PatchMerging
    global i


    layernorm_idx = 1
    mlp_layer_idx = 1

    if isinstance(outer_layer, SwinTransformerBlock3D):
        for inner_layer in outer_layer.layers:
            print(inner_layer)
            # Layer norm.
            if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                #print("layer norm")
                layer_norm_prefix = (
                    f"norm{layernorm_idx}"
                )
                inner_layer.gamma.assign(
                    tf.Variable(
                        np_state_dict[f"{layer_norm_prefix}.weight"]
                    )
                )
            #   print(i)
            #   i += 1

                model_layers[f"{layer_norm_prefix}.weight"] = inner_layer.gamma.name 


                inner_layer.beta.assign(
                    tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                )
            #   print(i)
            #   i += 1  
                model_layers[f"{layer_norm_prefix}.bias"] = inner_layer.beta.name 
                layernorm_idx += 1

            # Window attention.
            elif isinstance(inner_layer, WindowAttention3D):
                #print("window attention")
                attn_prefix = f"attn"

                # Relative position.
                inner_layer.relative_position_bias_table = (
                    modify_tf_block(
                        inner_layer.relative_position_bias_table,
                        np_state_dict[
                            f"{attn_prefix}.relative_position_bias_table"
                        ] ,
                        wn =f"{attn_prefix}.relative_position_bias_table" ,
                    )
                )
                inner_layer.relative_position_index = (
                    modify_tf_block(
                        inner_layer.relative_position_index,
                        np_state_dict[
                            f"{attn_prefix}.relative_position_index"
                        ],
                        wn = f"{attn_prefix}.relative_position_index"
                    )
                )

                # QKV.
                inner_layer.qkv = modify_tf_block(
                    inner_layer.qkv,
                    np_state_dict[f"{attn_prefix}.qkv.weight"],
                    np_state_dict[f"{attn_prefix}.qkv.bias"],
                    wn = f"{attn_prefix}.qkv.weight",
                    bn = f"{attn_prefix}.qkv.bias"
                )

                # Projection.
                inner_layer.proj = modify_tf_block(
                    inner_layer.proj,
                    np_state_dict[f"{attn_prefix}.proj.weight"],
                    np_state_dict[f"{attn_prefix}.proj.bias"],
                    wn = f"{attn_prefix}.proj.weight",
                    bn = f"{attn_prefix}.proj.bias"
                )

            # MLP.
            elif isinstance(inner_layer, tf.keras.Model):
                #print("mlp")
                mlp_prefix = f"mlp"
                for mlp_layer in inner_layer.layers:
                    if isinstance(mlp_layer, tf.keras.layers.Dense):
                        mlp_layer = modify_tf_block(
                            mlp_layer,
                            np_state_dict[
                                f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                            ],
                            np_state_dict[
                                f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                            ],
                            wn =  f"{mlp_prefix}.fc{mlp_layer_idx}.weight" ,
                            bn =  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                        )
                        mlp_layer_idx += 1


    return outer_layer


In [27]:
_ = modify_block3D(np_state_dict, "",b4_3d_tf )

In [28]:
out_pt, layers_3d_pt = b4_3d_pt(x_pt, mask)
out_tf, layers_3d_tf = b4_3d_tf(x_tf)


out_tf[:1,:1,:1,:1,:10], out_pt[:1,:1,:1,:1,:10]

window_partition torch.Size([2, 16, 7, 7, 768]) (8, 7, 7)
get window size (8, 7, 7) (4, 0, 0) (16, 7, 7) (8, 7, 7) (4, 3, 3) (<tf.Tensor: shape=(), dtype=int32, numpy=16>, <tf.Tensor: shape=(), dtype=int32, numpy=7>, <tf.Tensor: shape=(), dtype=int32, numpy=7>)
window_partition (2, 16, 7, 7, 768) (8, 7, 7)
before roll (4, 0, 0)


(<tf.Tensor: shape=(1, 1, 1, 1, 10), dtype=float32, numpy=
 array([[[[[0.39841163, 0.29438782, 0.7648544 , 0.18392481, 0.39647135,
            0.24303995, 0.6984857 , 0.18271445, 0.24370697, 0.5194623 ]]]]],
       dtype=float32)>,
 tensor([[[[[0.3984, 0.2944, 0.7649, 0.1839, 0.3965, 0.2430, 0.6985, 0.1827,
             0.2437, 0.5195]]]]]))

In [29]:
rtol = 1e-4
etol = 1e-4

for layer in layers_3d_pt:
    # print(layer)
    try:
        print( np.sum(np.absolute(layers_3d_tf[layer].numpy() - layers_3d_pt[layer].detach().numpy())), layer)
    except:
        print("---- ",layers_3d_tf[layer] , layers_3d_pt[layer], "\n")

    # np.testing.assert_allclose(layers_3d_tf[layer].numpy(), layers_3d_pt[layer].detach().numpy(), etol, rtol)


0.13432789 paddings
0.13432789 shifted_x
0.0 attn_mask
0.030836979 shifted_x final
0.030836979 attn_windows
0.13432789 x_windowns
0.030836973 after roll
0.030836973 forward1 output
0.18754846 norm2
0.15439126 mlp
0.15439126 drop_path


In [20]:
rtol = 1e-4
etol = 1e-4

print(np.sum(np.absolute(out_tf.numpy() - out_pt.detach().numpy())))

np.testing.assert_allclose(out_tf.numpy(), out_pt.detach().numpy(), etol, rtol)


13256.748


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.0001

Mismatched elements: 1193553 / 1204224 (99.1%)
Max absolute difference: 0.07644612
Max relative difference: 12859.115
 x: array([[[[[ 4.700059e-01,  1.364274e-01,  5.944030e-01, ...,
            6.148278e-01,  6.640177e-01,  5.569411e-01],
          [ 5.958239e-01,  3.900994e-02,  1.867094e-01, ...,...
 y: array([[[[[ 4.546654e-01,  1.445620e-01,  5.644763e-01, ...,
            5.853993e-01,  6.846541e-01,  5.636136e-01],
          [ 6.108807e-01,  3.019743e-02,  2.369033e-01, ...,...

0.13693267 paddings
1390012.8 shifted_x
0.0 attn_mask
10839.422 shifted_x final
10839.422 attn_windows
1390012.8 x_windowns
499.63043 after roll
499.63043 forward1 output
1718.5707 norm2
350.93048 mlp
350.93048 drop_path


# Basic layer 4

In [ ]:
x_tf, x_pt = get_x((2,768,16,7,7))


In [ ]:
from traceback import print_tb
from scipy.fftpack import shift
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.ops.gen_math_ops import imag



class BasicLayer(tf.keras.Model):


    def __init__(self,dim,shape_of_input,depth,num_heads,window_size=(1,7,7),mlp_ratio=4.,qkv_bias=False,qk_scale=None,drop=0.,attn_drop=0.,drop_path=0.,norm_layer=LayerNormalization,downsample=None,use_checkpoint=False):
        super().__init__()

        print(self, dim, shape_of_input,depth,num_heads,window_size, mlp_ratio,qkv_bias,qk_scale,drop,attn_drop,drop_path,norm_layer,downsample,use_checkpoint )

        self.window_size = window_size
        self.shift_size = tuple(i // 2 for i in window_size)
        self.depth = depth
        self.use_checkpoint = use_checkpoint
        self.shape_of_input = shape_of_input

        self.compute_mask_info = {
            "shape_of_input": self.shape_of_input,
            "window_size": self.window_size, 
            "shift_size": self.shift_size
        }

        # build 
        self.blocks = [
            SwinTransformerBlock3D(
                dim=dim,
                compute_mask_info = self.compute_mask_info,
                num_heads=num_heads,
                window_size=window_size,
                shift_size=(0,0,0) if (i % 2 == 0) else self.shift_size,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop,
                attn_drop=attn_drop,
                drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                norm_layer=norm_layer,
                use_checkpoint=use_checkpoint,
            )
            for i in range(depth)]
        
        self.downsample = downsample
        if self.downsample is not None:
            self.downsample = downsample(dim=dim, norm_layer=norm_layer, shape_of_input = self.compute_mask_info['shape_of_input'])

        self.outputs = {}

    def call(self, x):
        """ Forward function.
        Args:
            x: Input feature, tensor size (B, C, D, H, W).
        """
        print(self.name, x.shape)

        # calculate attention mask for SW-MSA
        B, C, D, H, W = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2] , tf.shape(x)[3] , tf.shape(x)[4] 

        x = tf.transpose(x, perm=[0, 2,3,4, 1 ])
        # print()
        # print("self.compute_mask_info", self.compute_mask_info)


        i = 1
        for blk in self.blocks:
            x = blk(x)
            self.outputs[f"block_{i}"] = x
            i += 1


        x = tf.reshape(x, [B, D, H, W, -1])


        if self.downsample is not None:
            x = self.downsample(x)
        x = tf.transpose(x, perm=[0, 4,1,2,3  ])

        return x ,self.outputs


In [ ]:
basic4_tf  = BasicLayer(768 ,(2, 768, 16, 7, 7) ,2, 24, (8, 7, 7), 4.0, True, None, 0.4, 0.0, [0.0, 0.0], tf.keras.layers.LayerNormalization , None ,False )
print("\n-------------\n")
basic4_pt = BasicLayer_pt(768 ,2, 24, (8, 7, 7), 4.0, True, None, 0.4, 0.0, [0.0, 0.0],  nn.LayerNorm , None ,False )

<__main__.BasicLayer object at 0x2b9eb8310> 768 (2, 768, 16, 7, 7) 2 24 (8, 7, 7) 4.0 True None 0.4 0.0 [0.0, 0.0] <class 'keras.layers.normalization.layer_normalization.LayerNormalization'> None False
3d block 768 {'shape_of_input': (2, 768, 16, 7, 7), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)} 24 (8, 7, 7) (0, 0, 0) 4.0 True None 0.4 0.0 0.0 <function gelu at 0x155687670> <class 'keras.layers.normalization.layer_normalization.LayerNormalization'> False
3d block 768 {'shape_of_input': (2, 768, 16, 7, 7), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)} 24 (8, 7, 7) (4, 3, 3) 4.0 True None 0.4 0.0 0.0 <function gelu at 0x155687670> <class 'keras.layers.normalization.layer_normalization.LayerNormalization'> False

-------------

3d block 768 24 (8, 7, 7) (0, 0, 0) 4.0 True None 0.4 0.0 0.0 <class 'torch.nn.modules.activation.GELU'> <class 'torch.nn.modules.normalization.LayerNorm'> False
3d block 768 24 (8, 7, 7) (4, 3, 3) 4.0 True None 0.4 0.0 0.0 <class 'torch.nn.modules.act

In [ ]:
out_pt , outputs_pt= basic4_pt(x_pt)
print("tf")
out_tf, outputs_tf = basic4_tf(x_tf)

out_tf[:1,:1,:1,:1,:10], out_pt[:1,:1,:1,:1,:10]

tf
basic_layer (2, 768, 16, 7, 7)
get window size (8, 7, 7) (0, 0, 0) (16, 7, 7) (8, 7, 7) (0, 0, 0) (<tf.Tensor: shape=(), dtype=int32, numpy=16>, <tf.Tensor: shape=(), dtype=int32, numpy=7>, <tf.Tensor: shape=(), dtype=int32, numpy=7>)


ValueError: Exception encountered when calling layer "swin_transformer_block3d_2" (type SwinTransformerBlock3D).

Attempt to convert a value (DictWrapper({'paddings': <tf.Tensor: shape=(2, 16, 7, 7, 768), dtype=float32, numpy=
array([[[[[-1.56123877e+00, -9.71423745e-01, -9.07378912e-01, ...,
            1.42002392e+00,  1.31023169e+00, -1.58783174e+00],
          [-2.42303610e-02, -1.67539823e+00, -7.07740545e-01, ...,
           -3.89452696e-01, -8.80255103e-01, -1.54426157e+00],
          [-8.83328199e-01,  5.42742491e-01,  5.54039478e-01, ...,
            5.35794735e-01, -9.46109295e-01,  1.41058087e-01],
          ...,
          [-1.56220686e+00, -2.10523725e-01, -3.07026625e-01, ...,
           -5.50640106e-01, -1.61959291e-01, -7.67081976e-02],
          [-1.63970828e+00,  2.50253677e-02, -1.38743424e+00, ...,
            2.20016122e-01,  1.65872037e+00,  1.30454981e+00],
          [-1.00278425e+00,  1.53643012e-01, -6.34205341e-03, ...,
           -1.42643666e+00,  1.15518689e+

In [ ]:
basic4_pt.eval()

np_state_dict = basic4_pt.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}
np_state_dict.keys()

dict_keys(['blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.relative_position_bias_table', 'blocks.0.attn.relative_position_index', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.relative_position_bias_table', 'blocks.1.attn.relative_position_index', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias'])

In [ ]:
model_layers = {}
def modify_basic_block(np_state_dict, pt_weights_prefix, tf_block):
  # PatchMerging
  global i
  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"
  
      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"] , wn = f"{patch_merging_idx}.reduction.weight")
      
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"],
                        
                        wn = f"{patch_merging_idx}.norm.weight",
                        bn = f"{patch_merging_idx}.norm.bias"
                        )
      
  # Swin Layers
      # Swin layers.
  common_prefix = f"blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  #print("layer norm")
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )
                #   print(i)
                #   i += 1

                  model_layers[f"{layer_norm_prefix}.weight"] = inner_layer.gamma.name 


                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )
                #   print(i)
                #   i += 1  
                  model_layers[f"{layer_norm_prefix}.bias"] = inner_layer.beta.name 
                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  #print("window attention")
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] ,
                          wn =f"{attn_prefix}.relative_position_bias_table" ,
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ],
                          wn = f"{attn_prefix}.relative_position_index"
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"],
                      wn = f"{attn_prefix}.qkv.weight",
                       bn = f"{attn_prefix}.qkv.bias"
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"],
                      wn = f"{attn_prefix}.proj.weight",
                      bn = f"{attn_prefix}.proj.bias"
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  #print("mlp")
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ],
                              wn =  f"{mlp_prefix}.fc{mlp_layer_idx}.weight" ,
                              bn =  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


In [ ]:
_ = modify_basic_block(np_state_dict,
                            f"",
                            basic4_tf.layers)

In [ ]:
out_pt , outputs_pt= basic4_pt(x_pt)
out_tf, outputs_tf = basic4_tf(x_tf)

out_tf[:1,:1,:1,:1,:10], out_pt[:1,:1,:1,:1,:10]

basic_layer (2, 768, 16, 7, 7)


(<tf.Tensor: shape=(1, 1, 1, 1, 7), dtype=float32, numpy=
 array([[[[[0.95478684, 0.20081165, 0.31327608, 0.8376813 , 0.8460306 ,
            0.9919535 , 0.17966008]]]]], dtype=float32)>,
 tensor([[[[[0.9554, 0.1998, 0.3136, 0.8380, 0.8468, 0.9917, 0.1808]]]]]))

In [ ]:
rtol = 1e-4
etol = 1e-4

for layer in outputs_tf:
    print(layer, outputs_tf[layer].shape)
    print( np.sum(np.absolute(outputs_tf[layer].numpy() - outputs_pt[layer].detach().numpy())), layer)

    np.testing.assert_allclose(outputs_tf[layer].numpy(), outputs_pt[layer].detach().numpy(), etol, rtol)


block_1 (2, 16, 7, 7, 768)
0.15911476 block_1
block_2 (2, 16, 7, 7, 768)
556.578 block_2


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.0001

Mismatched elements: 944918 / 1204224 (78.5%)
Max absolute difference: 0.00364685
Max relative difference: 228.66853
 x: array([[[[[ 9.547868e-01,  9.218199e-01,  8.107113e-01, ...,
            8.690614e-01,  8.275738e-01,  5.726429e-01],
          [ 2.008117e-01,  8.108054e-01,  7.693000e-01, ...,...
 y: array([[[[[ 9.553586e-01,  9.199904e-01,  8.102056e-01, ...,
            8.694789e-01,  8.267415e-01,  5.734892e-01],
          [ 1.997986e-01,  8.108619e-01,  7.697344e-01, ...,...

# Patch Embed

In [ ]:
pt_model.eval()
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

## Conv3D

In [ ]:
patch_size = (2, 4,4)
embed_dim = 96
window_size = (8, 7, 7)
layer_norm = tf.keras.layers.LayerNormalization

In [ ]:
tf_conv3D = tf.keras.layers.Conv3D(embed_dim, kernel_size = patch_size, strides= patch_size)

In [ ]:
pt_conv3D = nn.Conv3d(3, embed_dim, kernel_size = patch_size, stride= patch_size)

In [ ]:
x_tf, x_pt = get_x()
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))

output_of_conv3d_tf = tf_conv3D(x_tf)
output_of_conv3d_pt = pt_conv3D(x_pt)

In [ ]:
pt_conv3D.eval()
np_state_dict = pt_conv3D.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}
np_state_dict.keys()

In [ ]:
tf_conv3D = modify_tf_block(tf_conv3D, np_state_dict["weight"], np_state_dict["bias"])
tf_conv3D.weights[0].shape

In [ ]:
x_tf, x_pt = get_x()
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))

In [ ]:
output_of_conv3d_tf = tf_conv3D(x_tf)
output_of_conv3d_tf = tf.transpose(output_of_conv3d_tf, perm=[0, 4, 1, 2,3 ])

output_of_conv3d_pt = pt_conv3D(x_pt)

In [ ]:
output_of_conv3d_tf[:1,:1,:1,:1,:10] , output_of_conv3d_pt[:1,:1,:1,:1,:10] 

In [ ]:
output_of_conv3d_tf[:1,:1,:1,:1,:10] , output_of_conv3d_pt[:1,:1,:1,:1,:10] 

In [ ]:
rtol = 1e-4
etol = 1e-4

np.testing.assert_allclose(output_of_conv3d_tf.numpy(), output_of_conv3d_pt.detach().numpy(), etol, rtol)

np.sum(np.absolute(output_of_conv3d_tf.numpy() - output_of_conv3d_pt.detach().numpy()))

## Norm

In [ ]:
tf_norm = layer_norm(epsilon = 1e-5)
pt_norm = nn.LayerNorm(embed_dim)


In [ ]:
B, C, D, Wh, Ww = output_of_conv3d_tf.shape
input_norm_tf  = tf.reshape(output_of_conv3d_tf, shape=[B, C, -1]) 
input_norm_tf = tf.transpose(input_norm_tf, perm=[0 , 2, 1]) 

norm_layer_output_tf = tf_norm(input_norm_tf)
norm_layer_output_tf   = tf.transpose(norm_layer_output_tf, perm=[0,2,1])
norm_layer_output_tf = tf.reshape(norm_layer_output_tf, shape=[-1, embed_dim, D, Wh, Ww])

_, _, D, Wh, Ww = output_of_conv3d_pt.size()
input_norm_pt =  output_of_conv3d_pt.flatten(2).transpose(1, 2)
norm_layer_output_pt = pt_norm(input_norm_pt)
print(norm_layer_output_pt.shape)
norm_layer_output_pt = norm_layer_output_pt.transpose(1, 2).view(-1, embed_dim, D, Wh, Ww)

norm_layer_output_pt.shape

In [ ]:
pt_norm.eval()
np_state_dict = pt_norm.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}
np_state_dict.keys()

In [ ]:
tf_norm = modify_tf_block(tf_norm, np_state_dict["weight"], np_state_dict["bias"])
tf_norm.weights[0].shape

In [ ]:
B, C, D, Wh, Ww = output_of_conv3d_tf.shape
norm_layer_output_tf = tf_norm(input_norm_tf)
norm_layer_output_tf   = tf.transpose(norm_layer_output_tf, perm=[0,2,1])
norm_layer_output_tf = tf.reshape(norm_layer_output_tf, shape=[-1, embed_dim, D, Wh, Ww])

_, _, D, Wh, Ww = output_of_conv3d_pt.size()
norm_layer_output_pt = pt_norm(input_norm_pt)
print(norm_layer_output_pt.shape)

norm_layer_output_pt = norm_layer_output_pt.transpose(1, 2).view(-1, embed_dim, D, Wh, Ww)

norm_layer_output_pt.shape

In [ ]:
rtol = 1e-4
etol = 1e-4
# asserts error at 1e-7

np.testing.assert_allclose(norm_layer_output_tf.numpy(), norm_layer_output_pt.detach().numpy(), etol, rtol)


## SwinTransformerBlock3D

In [ ]:
dim= 96
compute_mask_info={'shape_of_input': (2, 96, 2, 56, 56), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)} 
num_heads=3
window_size=   (8, 7, 7) 
shift_size= (0, 0, 0)
mlp_ratio=  4.0
qkv_bias= True
qk_scale= None
drop= 0.0
attn_drop= 0.0
drop_path = 0.0
act_layer = tf.keras.activations.gelu 
norm_layer = tf.keras.layers.LayerNormalization
use_checkpoint=False


define input

In [ ]:
basic_layer_input_tf = tf.transpose(norm_layer_output_tf, perm=[0, 2,3,4, 1 ])

# norm_layer_output_pt= norm_layer_output_pt.contiguous()

B, C, D, H, W = norm_layer_output_pt.shape
basic_layer_input_pt = rearrange(norm_layer_output_pt, 'b c d h w -> b d h w c ')
basic_layer_input_pt.shape

In [ ]:
basic_layer_input_tf, basic_layer_input_pt = get_x((2,4,56,56,96))
basic_layer_input_tf.shape

In [ ]:
Dp = int(np.ceil(D / window_size[0])) * window_size[0]
Hp = int(np.ceil(H / window_size[1])) * window_size[1]
Wp = int(np.ceil(W / window_size[2])) * window_size[2]
attn_mask_pt = compute_mask_pt(Dp, Hp, Wp, window_size, shift_size, basic_layer_input_pt.device)

attn_mask_pt.shape, Dp, Hp, Wp

In [ ]:
##@title SwinTransformerBlock3D tf

from tensorflow.keras.layers import LayerNormalization
import numpy as np

class SwinTransformerBlock3D(tf.keras.Model):
    def __init__(self, dim, compute_mask_info, num_heads, window_size=(2,7,7), shift_size=(0,0,0),
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=tf.keras.activations.gelu, norm_layer=LayerNormalization, use_checkpoint=False):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        self.use_checkpoint=use_checkpoint
        self.compute_mask_info = compute_mask_info

        self.layer_wise_output = {}
        
        assert 0 <= self.shift_size[0] < self.window_size[0], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[1] < self.window_size[1], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[2] < self.window_size[2], "shift_size must in 0-window_size"

        self.norm1 = norm_layer(axis=-1, epsilon=1e-5)

        self.attn = WindowAttention3D(
            dim, window_size=self.window_size, num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        # compute mask
        print(compute_mask_info)
        mask_window_size, mask_shift_size = get_window_size((4,56,56), self.compute_mask_info["window_size"], self.compute_mask_info["shift_size"])
        

        Dp = int(tf.math.ceil(D/ mask_window_size[0])) * mask_window_size[0]
        Hp = int(tf.math.ceil(H / mask_window_size[1])) * mask_window_size[1]
        Wp = int(tf.math.ceil(W / mask_window_size[2])) * mask_window_size[2]

        self.attn_mask = compute_mask(Dp, Hp, Wp, mask_window_size, mask_shift_size)
        print(self.attn_mask.shape)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else tf.identity
        self.norm2 = norm_layer(epsilon=1e-5)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = mlp_block(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward_part1(self, x):
        
        B, D, H, W, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2] , tf.shape(x)[3] , tf.shape(x)[4] 
        b, c, d, h ,w = self.compute_mask_info['shape_of_input']

        #####
        print("window pt",self.window_size, self.shift_size)
        window_size, shift_size = get_window_size((D , H, w), self.window_size, self.shift_size)
        print(window_size, shift_size)

        x = self.norm1(x)
        self.layer_wise_output["norm1"] = x[:]
        print("norm", x.shape)

        pad_l = pad_t = pad_d0 = 0
        pad_d1 = (window_size[0] - D % window_size[0]) % window_size[0]
        pad_b = (window_size[1] - H % window_size[1]) % window_size[1]
        pad_r = (window_size[2] - W % window_size[2]) % window_size[2]
        paddings = [[0,0] , [pad_d0, pad_d1] , [pad_t, pad_b] , [pad_l, pad_r], [0, 0] ]

        x = tf.pad(x, paddings)
        self.layer_wise_output["pad_feature_map"] = x[:]
        print("padding", x.shape)


        _, Dp, Hp, Wp, _ = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2] , tf.shape(x)[3] , tf.shape(x)[4] 

        # cyclic shift
        # print(self.shift_size)
        if any(i > 0 for i in self.shift_size):
            shifted_x = tf.roll(x, shift=[-self.shift_size[0], -self.shift_size[1], -self.shift_size[2]], axis=[1, 2, 3]) #?
            attn_mask = self.attn_mask
            print("inside any cyclic shift")
        else:
            shifted_x = x
            attn_mask = None

        print("shifted_x", x.shape)
        self.layer_wise_output["shifted_x"] = shifted_x[:]
        self.layer_wise_output["attn_mask"] = attn_mask

        print("window_partion",shifted_x.shape, window_size)
        x_windows = window_partition(shifted_x, window_size)  # B*nW, Wd*Wh*Ww, C
        
        self.layer_wise_output["partion_windows"] = x_windows[:]

        attn_windows = self.attn(x_windows, mask=attn_mask)  # B*nW, Wd*Wh*Ww, C
        attn_windows = tf.reshape( attn_windows ,  [-1, *(window_size+(C,))] )
        shifted_x = window_reverse(attn_windows, window_size, B, Dp, Hp, Wp)  # B D' H' W' C

        if any(i > 0 for i in self.shift_size):
            x = tf.roll(shifted_x, shift=[self.shift_size[0], self.shift_size[1], self.shift_size[2]], axis=[1, 2, 3]) #?
        else:
            x = shifted_x
        self.layer_wise_output["reverse cyclic shift"] = x[:]
        

        if pad_d1 >0 or pad_r > 0 or pad_b > 0:
            x = x[:, :D, :H, :W, :]
        return x

    def forward_part2(self, x):
        return self.drop_path(self.mlp(self.norm2(x)))

    def call(self, x):
        layer_wise_output = {}

        shortcut = x
        x = self.forward_part1(x)
        x = shortcut + self.drop_path(x)
        x = x + self.forward_part2(x)
        return x

In [ ]:
swin_block3d_tf_1 = SwinTransformerBlock3D(dim, compute_mask_info, num_heads, window_size, shift_size,
                 mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path,
                 act_layer, norm_layer, use_checkpoint)

In [ ]:
#@title SwinTransformerBlock3D pt

class SwinTransformerBlock3D_pt(nn.Module):

    def __init__(self, dim, num_heads, window_size=(2,7,7), shift_size=(0,0,0),
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm, use_checkpoint=False):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        self.use_checkpoint=use_checkpoint

        self.layer_wise_output = {}
        assert 0 <= self.shift_size[0] < self.window_size[0], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[1] < self.window_size[1], "shift_size must in 0-window_size"
        assert 0 <= self.shift_size[2] < self.window_size[2], "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention3D_pt(
            dim, window_size=self.window_size, num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath_pt(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp_pt(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward_part1(self, x, mask_matrix):
        B, D, H, W, C = x.shape
        print("window pt",self.window_size, self.shift_size)
        print(D,H,W)
        window_size, shift_size = get_window_size_pt((D, H, W), self.window_size, self.shift_size)
        print(window_size, shift_size)
        
        x = self.norm1(x)

        self.layer_wise_output["norm1"] = x[:]

        # pad feature maps to multiples of window size
        pad_l = pad_t = pad_d0 = 0
        pad_d1 = (window_size[0] - D % window_size[0]) % window_size[0]
        pad_b = (window_size[1] - H % window_size[1]) % window_size[1]
        pad_r = (window_size[2] - W % window_size[2]) % window_size[2]
        x = F.pad(x, (0, 0, pad_l, pad_r, pad_t, pad_b, pad_d0, pad_d1))

        self.layer_wise_output["pad_feature_map"] = x[:]


        _, Dp, Hp, Wp, _ = x.shape
        # cyclic shift
        if any(i > 0 for i in shift_size):
            shifted_x = torch.roll(x, shifts=(-shift_size[0], -shift_size[1], -shift_size[2]), dims=(1, 2, 3))
            attn_mask = mask_matrix
        else:
            print("else",x.shape)
            shifted_x = x
            attn_mask = None

        self.layer_wise_output["shifted_x"] = shifted_x[:]
        self.layer_wise_output["attn_mask"] = attn_mask
        # partition windows
        print("window_partion",shifted_x.shape, window_size)
        x_windows = window_partition_pt(shifted_x, window_size)  # B*nW, Wd*Wh*Ww, C
        self.layer_wise_output["partion_windows"] = x_windows[:]
        
        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=attn_mask)  # B*nW, Wd*Wh*Ww, C
        # merge windows
        attn_windows = attn_windows.view(-1, *(window_size+(C,)))
        shifted_x = window_reverse_pt(attn_windows, window_size, B, Dp, Hp, Wp)  # B D' H' W' C
        # reverse cyclic shift
        if any(i > 0 for i in shift_size):
            x = torch.roll(shifted_x, shifts=(shift_size[0], shift_size[1], shift_size[2]), dims=(1, 2, 3))
        else:
            x = shifted_x
        self.layer_wise_output["reverse cyclic shift"] = x[:]

        if pad_d1 >0 or pad_r > 0 or pad_b > 0:
            x = x[:, :D, :H, :W, :].contiguous()
        return x

    def forward_part2(self, x):
        return self.drop_path(self.mlp(self.norm2(x)))

    def forward(self, x, mask_matrix):


        shortcut = x
        if self.use_checkpoint:
            x = checkpoint.checkpoint(self.forward_part1, x, mask_matrix)
        else:
            x = self.forward_part1(x, mask_matrix)

        x = shortcut + self.drop_path(x)

        if self.use_checkpoint:
            x = x + checkpoint.checkpoint(self.forward_part2, x)
        else:
            x = x + self.forward_part2(x)

        return x

In [ ]:
swin_block3d_pt_1 = SwinTransformerBlock3D_pt(dim , num_heads, window_size, shift_size)

In [ ]:
swin_block3d_tf_1_output = swin_block3d_tf_1(basic_layer_input_tf)
print()
swin_block3d_pt_1_output = swin_block3d_pt_1(basic_layer_input_pt, attn_mask_pt)

swin_block3d_tf_1_output[:1,:1,:1,:1,:10], swin_block3d_pt_1_output[:1,:1,:1,:1,:10] 

In [ ]:
swin_block3d_tf_1_output = swin_block3d_tf_1(basic_layer_input_tf)
print()
swin_block3d_pt_1_output = swin_block3d_pt_1(basic_layer_input_pt, attn_mask_pt)

swin_block3d_tf_1_output[:1,:1,:1,:1,:10], swin_block3d_pt_1_output[:1,:1,:1,:1,:10] 

In [ ]:
swin_block3d_pt_1.eval()
np_state_dict = swin_block3d_pt_1.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}
np_state_dict.keys()


In [ ]:
swinBlockWeights = {}
ptModelWeights = pt_model.state_dict()
for layer in ptModelWeights:
  if "layers.0.blocks.0" in layer:
    swinBlockWeights[layer[18:]] = ptModelWeights[layer]
swinBlockWeights.keys()

In [ ]:
swin_block3d_pt_1.load_state_dict(swinBlockWeights)

In [ ]:
_ = modify_swin_block3D(np_state_dict,
                            f"",
                            swin_block3d_tf_1)

In [ ]:
swin_block3d_tf_1_output = swin_block3d_tf_1(basic_layer_input_tf)
swin_block3d_pt_1_output = swin_block3d_pt_1(basic_layer_input_pt, attn_mask_pt)

swin_block3d_tf_1_output[:1,:1,:1,:1,:10], swin_block3d_pt_1_output[:1,:1,:1,:1,:10] 

In [ ]:
block3D_layers_output_tf = swin_block3d_tf_1.layer_wise_output
block3D_layers_output_pt = swin_block3d_pt_1.layer_wise_output
print(block3D_layers_output_pt["attn_mask"])

In [ ]:
rtol = 1e-4
etol = 1e-4
# asserts error at 1e-7
for layer in block3D_layers_output_pt:
  print("\n",layer, end="")
  if block3D_layers_output_tf[layer] is None and block3D_layers_output_pt[layer] is None:
    continue
  np.testing.assert_allclose(block3D_layers_output_tf[layer].numpy(), block3D_layers_output_pt[layer].detach().numpy(), etol, rtol)
  print(" has passed the test")

In [ ]:
rtol = 1e-4
etol = 1e-4
# asserts error at 1e-7

np.testing.assert_allclose(swin_block3d_tf_1_output.numpy(), swin_block3d_pt_1_output.detach().numpy(), etol, rtol)


In [ ]:
import tensorflow as tf
def get_window_size(x_size, window_size, shift_size=None):
    #print("get window",x_size, window_size, shift_size)
    use_window_size = list(window_size)
    if shift_size is not None:
        use_shift_size = list(shift_size)
    for i in range(len(x_size)):
        if x_size[i] <= window_size[i]:
            use_window_size[i] = x_size[i]
            if shift_size is not None:
                use_shift_size[i] = 0



    if shift_size is None:
        return tuple(use_window_size)
    else:
        return tuple(use_window_size), tuple(use_shift_size)

In [ ]:
get_window_size((4,56,56), (8,7,7), (0,0,0))

In [ ]:
import tensorflow as tf
def get_window_size(x_size, window_size, shift_size=None):
    #print("get window",x_size, window_size, shift_size)

    print("get_window_size parameters",x_size, window_size, shift_size)

    use_window_size = list(window_size)
    if shift_size is not None:
        use_shift_size = list(shift_size)
    for i in range(len(x_size)):
        if x_size[i] <= window_size[i]:
            use_window_size[i] = x_size[i]
            if shift_size is not None:
                use_shift_size[i] = 0



    if shift_size is None:
        return tuple(use_window_size)
    else:
        print(tuple(use_window_size), tuple(use_shift_size))
        return tuple(use_window_size), tuple(use_shift_size)

In [ ]:
get_window_size((4,56,56), (8,7,7), (4,3,3))


# Windows Partition

In [ ]:
(2, 16, 7, 7, 768) (8, 7, 7)

In [ ]:
x_tf , x_pt = get_x((2,16,7,7,768))

In [ ]:
import tensorflow as tf
from functools import reduce


def window_partition(x, window_size):
    """
    Args:
        x: (B, D, H, W, C)
        window_size (tuple[int]): window size
    Returns:
        windows: (B*num_windows, window_size*window_size, C)
    """
    B, D, H, W, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], tf.shape(x)[3] , tf.shape(x)[4] 
    
    x = tf.reshape(x, [B, D // window_size[0], window_size[0], H // window_size[1], window_size[1], W // window_size[2], window_size[2], C])

    windows = tf.reshape(tf.transpose(x, perm=[0, 1, 3, 5, 2, 4, 6, 7]), [-1, reduce((lambda x, y: x * y), window_size), C])  
                             
                                               
    return windows

In [ ]:
y= window_partition(x_tf, (8,7,7))
z = window_partition_pt(x_pt, (8,7,7))


window_partition torch.Size([2, 16, 7, 7, 768]) (8, 7, 7)


In [ ]:
print( np.sum(np.absolute(y.numpy() - z.detach().numpy())))


0.0


# cehck attention

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
import numpy as np

class WindowAttention3D(tf.keras.layers.Layer):
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        print("win attn params",dim, window_size, num_heads)

        self.qkv = Dense(dim * 3, use_bias=qkv_bias)
        self.attn_drop = Dropout(attn_drop)
        self.proj = Dense(dim)
        self.proj_drop = Dropout(proj_drop)


    def build(self, shape_of_input):
        self.relative_position_bias_table = self.add_weight(shape=((2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1)* (2 * self.window_size[2] - 1), self.num_heads),
                                                            initializer=tf.initializers.Zeros(), trainable=True, name="relative_position_bias_table") # 2*Wd-1 * 2*Wh-1 * 2*Ww-1, nH

        coords_d = np.arange(self.window_size[0])
        coords_h = np.arange(self.window_size[1])
        coords_w = np.arange(self.window_size[2])
        coords = np.stack(np.meshgrid(coords_d, coords_h, coords_w, indexing='ij')) # 3, Wd, Wh, Ww
        coords_flatten = coords.reshape(3, -1)
        relative_coords = coords_flatten[:, :,
                                         None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 2] += self.window_size[2] - 1

        relative_coords[:, :, 0] *= (2 * self.window_size[1] - 1) * (2 * self.window_size[2] - 1)
        relative_coords[:, :, 1] *= (2 * self.window_size[2] - 1)
        relative_position_index = relative_coords.sum(-1).astype(np.int64)
        

        self.relative_position_index = tf.Variable(initial_value=tf.convert_to_tensor(
            relative_position_index), trainable=False)
        self.built = True


    def call(self, x, mask=None):
        # #print(x.shape, "attention")
        B_, N, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        qkv = tf.transpose(tf.reshape(self.qkv(
            x), shape=[-1, N, 3, self.num_heads, C // self.num_heads]), perm=[2, 0, 3, 1, 4])
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = (q @ tf.transpose(k, perm=[0, 1, 3, 2]))
        index = tf.reshape(self.relative_position_index[:N, :N], shape=[-1])
        relative_position_bias = tf.gather(self.relative_position_bias_table, index)
        relative_position_bias = tf.reshape(relative_position_bias, shape=[N, N, -1])
        relative_position_bias = tf.transpose(relative_position_bias, perm=[2, 0, 1])
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:


            nW = tf.shape(mask)[0]  # tf.shape(mask)[0]
            attn = tf.reshape(attn, shape=[-1, nW, self.num_heads, N, N]) + tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), attn.dtype)

            attn = tf.reshape(attn, shape=[-1, self.num_heads, N, N])
            attn = tf.nn.softmax(attn, axis=-1)
        else:
            attn = tf.nn.softmax(attn, axis=-1)

        attn = self.attn_drop(attn)

        x = tf.transpose((attn @ v), perm=[0, 2, 1, 3])
        x = tf.reshape(x, shape=[-1, N, C])
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [ ]:
atten = WindowAttention3D(96, (8,7,7),3 )

In [ ]:
x_tf, x_pt = get_x((64,196,96))

In [ ]:
atten(x_tf, tf.zeros((64,98,98)))

In [ ]:
atten = WindowAttention3D_pt(96, (8,7,7),3 )
atten(x_pt, torch.zeros((64,98,98)))


# practice